# FOSDEM 2018, Bruxelles, Stefan Behnel

See http://consulting.behnel.de/

In [ ]:
%load_ext cython

In [ ]:
import sys
import Cython
import numpy as np
import matplotlib
import subprocess, os
print("Python %d.%d.%d %s %s" % sys.version_info)
print("Cython %s" % Cython.__version__)
print("NumPy  %s" % np.__version__)
print("Matplotlib  %s" % matplotlib.__version__)
print(subprocess.check_output([os.environ.get('CC', 'cc'), "--version"]).decode().splitlines()[0])
print([line for line in subprocess.check_output([os.environ.get('CC', 'cc'), "--version", "-v"],
                                                stderr=subprocess.STDOUT).decode().splitlines()
       if ' version ' in line][0])

In [ ]:
%matplotlib inline

## Cython Intro

In [ ]:
from math import sin
sin(5)

In [ ]:
%%cython
from math import sin
print(sin(5))

In [ ]:
%%cython
cimport libc.math
sin_func = libc.math.sin

In [ ]:
sin_func(5)

In [ ]:
%%cython
cimport libc.math

def csin(double x):
    return libc.math.sin(x)

In [ ]:
csin(5)

In [ ]:
%%cython -a
cimport libc.math

def csin(double x):
    return libc.math.sin(x)

In [ ]:
%%cython
cimport libc.math

def square_sin(double x):
    cdef double x_square = x * x
    return libc.math.sin(x_square)

In [ ]:
square_sin(5)

## A bit of C memory handling

In [ ]:
%%cython
# dynamic C memory allocation
from libc.stdlib cimport malloc, free

cdef int* cmem = <int*>malloc(22 * sizeof(int))
if not cmem:
    raise MemoryError()
try:
    cmem[:2] = [42, 21]
    print(cmem[1])
finally:
    free(cmem)

## Using native third-party libraries

In [ ]:
%%cython
# distutils: include_dirs=/usr/include/luajit-2.0
# distutils: libraries=luajit-5.1

## distutils: include_dirs=/usr/include/lua5.1
## distutils: libraries=lua5.1

cdef extern from "lua.h":
    ctypedef struct lua_State
    lua_State *luaL_newstate ()
    void lua_close (lua_State *L)
    int luaL_loadbuffer (lua_State *L, char *buff, size_t sz, char *name)
    void lua_settop (lua_State *L, int idx)
    int lua_gettop (lua_State *L)
    int lua_pcall (lua_State *L, int nargs, int nresults, int errfunc)
    int lua_type (lua_State *L, int idx)
    float lua_tonumber (lua_State *L, int idx)
    enum:
        LUA_TNUMBER
        LUA_MULTRET


def run_lua(code):
    cdef int result_status
    cdef float result

    if isinstance(code, unicode):
        code = code.encode('utf8')
    elif not isinstance(code, bytes):
        raise ValueError("code must be a string")

    # init Lua runtime
    L = luaL_newstate()
    if not L:
        raise MemoryError()

    try:
        # compile Lua code
        if luaL_loadbuffer(L, code, len(code), '<python>'):
            raise SyntaxError()

        # execute code
        if lua_pcall(L, 0, LUA_MULTRET, 0):
            raise RuntimeError()

        # convert return value (Lua number == float)
        assert lua_type(L, 1) == LUA_TNUMBER, "not a numeric return value"
        return lua_tonumber(L, 1)
    finally:
        lua_settop(L, 0)
        lua_close(L)

In [ ]:
code = '''
function fib(i)
    if i > 2 then
        return fib(i-1) + fib(i-2)
    else
        return 1
    end
end
'''

run_lua(code + "return fib(10)")

In [ ]:
%%timeit bench = code + "return fib(24)"

run_lua(bench)

## Everyone likes taxes !

Idea borrowed from Caleb Hattingh, PyCon-AU 2015,
http://pyvideo.org/pycon-au-2015/easy-wins-with-cython-fast-and-multi-core.html

In [ ]:
PEOPLE = 44_000_000
AVERAGE = 3703*12
print("Average income of {:,d} earners, Deutschland 2016: {:,d} €".format(PEOPLE, AVERAGE))

In [ ]:
# lacking offical data, let's create some alternative facts
import numpy as np
mu, sigma = 10.64, .35
s = np.random.lognormal(mu, sigma, PEOPLE // 20)
['{:,.2f} €'.format(x) for x in (np.min(s), np.mean(s), np.max(s))]

In [ ]:
import matplotlib.pyplot as plt
count, bins, ignored = plt.hist(s[s < 110000], 100, normed=True, align='mid')
x = np.linspace(min(bins), max(bins), 101)
pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))
       / (x * sigma * np.sqrt(2 * np.pi)))
plt.plot(bins, pdf, linewidth=2, color='r')
plt.axis('tight')
plt.show()

## Let's calculate everyone's taxes

https://de.wikipedia.org/wiki/Einkommensteuer_%28Deutschland%29#Tarif_2017

In [ ]:
# from Wikipedia:
# =WENN(A1>256303; A1*0,45-16164,53;
#  WENN(A1>54057; A1*0,42-8475,44;
#  WENN(A1>13769; (A1-13769)*((A1-13769)*0,0000022376+0,2397)+939,57;
#  WENN(A1>8820; (A1-8820)*((A1-8820)*0,0000100727+0,14); 0))))

def calculate_tax(income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_income(incomes):
    return sum(incomes) / len(incomes)

def average_tax_rate(incomes):
    return sum(calculate_tax(x) for x in incomes) / sum(incomes)

In [ ]:
AVERAGE, calculate_tax(AVERAGE)

In [ ]:
incomes_np = s
incomes = list(s)

In [ ]:
avg_in, avg_tax = average_income(incomes), average_tax_rate(incomes)
avg_in, avg_tax

In [ ]:
%%timeit
# NOTE: don't forget to disable Energy Management on laptops to get better timings!
average_tax_rate(incomes)

In [ ]:
Tpython = 1

## Making things comparable

In [ ]:
import operator

timings = {}

def ratios(**new):
    assert len(new) == 1
    timings.update(**new)
    last = list(new.values())[0]
    print('\n'.join('%10s: %7.2f' % (name, t / last)
                    for name, t in sorted(timings.items(), key=operator.itemgetter(1))))

ratios(python=Tpython)

## NumPy slicing

In [ ]:
# from Wikipedia:
# =WENN(A1>256303; A1*0,45-16164,53;
#  WENN(A1>54057; A1*0,42-8475,44;
#  WENN(A1>13769; (A1-13769)*((A1-13769)*0,0000022376+0,2397)+939,57;
#  WENN(A1>8820; (A1-8820)*((A1-8820)*0,0000100727+0,14); 0))))

def calculate_tax_numpy_segments(d):
    tax_seg1 = d[(d > 256303)] * 0.45 - 16164.53
    tax_seg2 = d[(d > 54057) & (d <= 256303)] * 0.42 - 8475.44
    seg3 = d[(d > 13769) & (d <= 54057)] - 13769
    seg4 = d[(d > 8820) & (d <= 13769)] - 8820
    prog_seg3 = seg3 * 0.0000022376 + 0.2397
    prog_seg4 = seg4 * 0.0000100727 + 0.14
    return (
        tax_seg1.sum() +
        tax_seg2.sum() +
        (seg3 * prog_seg3 + 939.57).sum() +
        (seg4 * prog_seg4).sum()
    ) / d.sum()


In [ ]:
incomes_np.mean(), calculate_tax_numpy_segments(incomes_np)

In [ ]:
%%timeit
calculate_tax_numpy_segments(incomes_np)

In [ ]:
ratios(numpy=1)

## NumPy ufunc

In [ ]:
calculate_tax_numpy = np.frompyfunc(calculate_tax, 1, 1)

In [ ]:
calculate_tax_numpy(incomes_np).sum() / incomes_np.sum()

In [ ]:
%%timeit
calculate_tax_numpy(incomes_np).sum() / incomes_np.sum()

In [ ]:
ratios(ufunc=1)

## Cython

In [ ]:
%%cython
# plain copy from Python code above, only renamed functions

def calculate_tax_cy(income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_income_cy(incomes):
    return sum(incomes) / len(incomes)

def average_tax_rate_cy(incomes):
    return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)

In [ ]:
average_income_cy(incomes), average_tax_rate_cy(incomes)

In [ ]:
%%timeit
average_tax_rate_cy(incomes)

In [ ]:
ratios(compiled=1)

## Faster Cython: static types

In [ ]:
%%cython -a

def calculate_tax_cy(income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_income(incomes):
    return sum(incomes) / len(incomes)

def average_tax_rate_cy(incomes):
    return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)

In [ ]:
average_tax_rate_cy(incomes)

In [ ]:
%%timeit
average_tax_rate_cy(incomes)

In [ ]:
ratios(typed=1)

In [ ]:
%%cython -a

# SOLUTION

cpdef double calculate_tax_cy(double income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_tax_rate_cy(incomes):
    # return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)
    cdef double tax = 0, income = 0, x
    for x in incomes:
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


In [ ]:
average_tax_rate_cy(incomes)

In [ ]:
%%timeit
average_tax_rate_cy(incomes)

In [ ]:
ratios(typed=1)

## Faster Cython: processing memory views

In [ ]:
%%cython

cimport cython

cpdef double calculate_tax_cy(double income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_tax_rate_memview(incomes):
    # return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)
    cdef double tax = 0, income = 0, x
    for x in incomes:
        income += x
        tax += calculate_tax_cy(x)
    return tax / income

In [ ]:
average_tax_rate_memview(incomes_np)

In [ ]:
%%timeit
average_tax_rate_memview(incomes_np)

In [ ]:
ratios(mviews=1)

In [ ]:
%%cython

# SOLUTION

cpdef double calculate_tax_cy(double income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_tax_rate_cy(incomes):
    # return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)
    cdef double tax = 0, income = 0, x
    for x in incomes:
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


cimport cython

@cython.boundscheck(False)
def average_tax_rate_memview(double[:] incomes):
    cdef unsigned long i
    cdef double tax = 0, income = 0, x
    for i in range(incomes.shape[0]):
        x = incomes[i]
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


In [ ]:
average_tax_rate_memview(incomes_np)

In [ ]:
%%timeit
average_tax_rate_memview(incomes_np)

In [ ]:
ratios(mviews=1)

## Faster Cython: prange

In [ ]:
%%cython
# cython: auto_pickle=False
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

cpdef double calculate_tax_cy(double income) nogil:
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

cimport cython
#from cython.parallel cimport prange

@cython.boundscheck(False)
def average_tax_rate_prange(double[:] incomes):
    cdef unsigned long i
    cdef double tax = 0, income = 0, x
    for i in range(len(incomes)):
        x = incomes[i]
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


In [ ]:
average_tax_rate_prange(incomes_np)

In [ ]:
%%timeit
average_tax_rate_prange(incomes_np)

In [ ]:
ratios(prange=1)

In [ ]:
%%cython
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

# SOLUTION

cpdef double calculate_tax_cy(double income) nogil:
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

cimport cython
from cython.parallel cimport prange

@cython.boundscheck(False)
def average_tax_rate_prange(double[:] incomes):
    cdef unsigned long i
    cdef double tax = 0, income = 0, x
    for i in prange(incomes.shape[0], nogil=True, num_threads=4):
        x = incomes[i]
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


In [ ]:
average_tax_rate_prange(incomes_np)

In [ ]:
%%timeit
average_tax_rate_prange(incomes_np)

In [ ]:
ratios(prange=1)

## Pythran integration

In [ ]:
%%cython -a --verbose
# cython: np_pythran=True

import numpy as np
cimport numpy as cnp

def calculate_tax_numpy_segments(cnp.ndarray[double, ndim=1] d):
    tax_seg1 = d[(d > 256303)] * 0.45 - 16164.53
    tax_seg2 = d[(d > 54057) & (d <= 256303)] * 0.42 - 8475.44
    seg3 = d[(d > 13769) & (d <= 54057)] - 13769
    seg4 = d[(d > 8820) & (d <= 13769)] - 8820
    prog_seg3 = seg3 * 0.0000022376 + 0.2397
    prog_seg4 = seg4 * 0.0000100727 + 0.14
    return (
        np.sum(tax_seg1) +
        np.sum(tax_seg2) +
        np.sum(seg3 * prog_seg3 + 939.57) +
        np.sum(seg4 * prog_seg4)
    ) / np.sum(d)
